# Run linetracker Example

This example shows how to use the both linetrackers and automoto on the board.  You are supposed to used arduino_automoto.ipynb and arduino_linetracker.ipynb separately before using this book.

For this notebook, a PYNQ Arduino is also required. Pins of device should be set the same in the previous notebooks.

In [1]:
# Make sure the base overlay is loaded
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

### 1. Instantiate and set the pins of automotos and linetrackers

In [2]:
from pynq.lib.arduino import Run_LT
from time import sleep

Before you use automoto, you can set the pins connected to arduino freely.

the PINS should obey the rules as followed:

PINS[0:4] is set the pins to control the pwm generate io for motor a,b,c,d in sequence

PINS[4:8] is to set the pin to control the direction of motor (1 normal, 0 reverse)

PINS[8:12] is to set the pins to record the velocity(FG)

 * auotmoto a: right front
 * auotmoto b: right rear
 * automoto c: left sacro-anterior / left front
 * auotmoto d: left rear

In [3]:
PINS = [6,9,10,11,7,8,12,13,4,5,18,19] #default pins of automoto
LT_PINS = ["CHANNEL_A0","CHANNEL_A1","CHANNEL_A2",
           "CHANNEL_A3","CHANNEL_A4","CHANNEL_A5"]

# Instantiate automoto on Arduino
run_lt = Run_LT(base.ARDUINO,"CHANNEL_A3", "CHANNEL_A2", PINS)
#for linetrackers
#default pins is left:CHANNEL_A3, right:CHANNEL_A2
#if you set the pins as the default configuration
#linetrack = LT_sen(base.ARDUINO) such a format is legal

[6, 9, 10, 11, 7, 8, 12, 13, 4, 5, 18, 19, 3, 2]


There are six moving model provided, each model can adjust velocity freely

In [4]:
DIRECTION = ["FORWARD","BACKWARD",
             "LEFT_S","RIGHT_S",
             "LEFT_R","RIGHT_R"]

### 2. Control the car to move by following the line

input the direction into run_lt.move() function, the default power is set as 50%

and the range of power is from 0 - 100 (unit: percent)

e.g.
* run_lt.move("FORWARD") legal
* run_lt.move("BACKWARD", 60) legal
* run_lt.move("backward") illegal
* run_lt.move("LEFT_S",101) illegal

The threshold can be set depend by your own.

In this example, the threshold is 0.65

In [10]:
while(1):
    data = run_lt.read_lt_data()
    if data[0] > 0.65 and data[1] < 0.65: 
        run_lt.move("LEFT_R")
        sleep(0.1)
        run_lt.move("FORWARD")
    elif data[1] > 0.65 and data[0] < 0.65:
        run_lt.move("RIGHT_R")
        sleep(0.1)
        run_lt.move("FORWARD")
    elif data[0] > 0.65 and data[1] > 0.65: 
        run_lt.stop()
        break
    else: run_lt.move("FORWARD")
print("Run to the terminal!")

Run to the terminal!


In [5]:
data = run_lt.read_lt_data()
print(data)

[0.06839263916015625, 0.4989207458496094, 1032521482, 1056916459]


In [11]:
run_lt.stop()